# Gigs Senior Data Analyst Challenge

Welcome to the Gigs data analyst take-home challenge! This notebook will help you get started with analyzing our connectivity usage data.

## About the Data

You'll be working with three main datasets:
- **Usage Data**: Detailed usage per subscription period (~100K+ records)
- **Plan Events**: Plan configuration and pricing history
- **Projects**: Project metadata

## Setup Instructions

Run the cells below to set up your environment and load the data into DuckDB.

In [1]:
# Import required libraries
import duckdb
import pandas as pd
from datetime import datetime, timedelta

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


In [4]:
# Load JupySQL extension and configure
%load_ext sql

# Configure JupySQL for better output
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

print("✅ JupySQL configured!")

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
✅ JupySQL configured!


In [5]:
# Connect to DuckDB
conn = duckdb.connect('gigs-analytics.db')
%sql conn --alias duckdb

print("✅ Connected to DuckDB database: gigs-analytics.db")

✅ Connected to DuckDB database: gigs-analytics.db


In [6]:
%%sql
-- Load data into DuckDB tables
CREATE OR REPLACE TABLE usage_data AS 
SELECT * FROM 'data/usage_by_subscription_period.csv';

CREATE OR REPLACE TABLE plan_events AS 
SELECT * FROM 'data/plan_change_events.csv';

CREATE OR REPLACE TABLE projects AS 
SELECT * FROM 'data/projects.csv';

,Count
0,3


In [7]:
%%sql
-- Verify data loading
select 
  'usage_data' as table_name, 
  count(*) as row_count,
  count(distinct subscription_id) as unique_subscriptions
from usage_data
union all
select 
  'plan_events' as table_name, 
  count(*) as row_count,
  count(distinct plan_id) as unique_plans
from plan_events
union all
select 
  'projects' as table_name, 
  count(*) as row_count,
  count(distinct project_id__hashed) as unique_projects
from projects;

,table_name,row_count,unique_subscriptions
0,usage_data,53565,8457
1,plan_events,209,36
2,projects,3,3


## Your Analysis Starts Here!

Now you have everything set up. Use the cells below to start your analysis.

### Tips:
- Use `%%sql` for multi-line SQL queries
- Use `%sql variable_name <<` to store results in a Python variable
- Combine SQL with Python/Pandas for advanced analysis
- Feel free to use any visualisation library you feel comfortable with

In [11]:
%%sql
select * from usage_data limit 5

,subscription_id,project_id__hashed,plan_id,reporting_date,subscription_period_start,subscription_period_end,subscription_period_number,cumulative_data_usage_megabyte,cumulative_voice_usage_minutes,cumulative_sms_usage,number_of_addons_activated
0,sub_b97107a1c7ef89cf28a7e83ee850,82728d5d3cf7f35b58bc318399c2c5caf7eeadcc37f359...,pln_0a3b55b9abb575b023c07e617b49,2024-02-07,2024-02-04,2024-02-07,1,0.000000,0.0,0.0,0
1,sub_2353ef9e8bf55916b97ceb30194c,82728d5d3cf7f35b58bc318399c2c5caf7eeadcc37f359...,pln_0a3b55b9abb575b023c07e617b49,2024-02-08,2024-01-25,2024-02-08,1,56.660992,1.0,1.0,0
2,sub_5d5605c8cf0b46116a42cedb9561,82728d5d3cf7f35b58bc318399c2c5caf7eeadcc37f359...,pln_1d3f49d61a672755efc1865c4d9a,2024-02-08,2024-01-25,2024-02-08,1,160.659456,0.0,1.0,0
3,sub_8e114d3a638d12ca1526a3fbedd6,82728d5d3cf7f35b58bc318399c2c5caf7eeadcc37f359...,pln_0a3b55b9abb575b023c07e617b49,2024-02-08,2024-01-28,2024-02-08,1,156.870656,93.0,198.0,0
4,sub_16059484727214a6b903fa16d0fe,82728d5d3cf7f35b58bc318399c2c5caf7eeadcc37f359...,pln_0a3b55b9abb575b023c07e617b49,2024-02-10,2024-02-09,2024-02-10,1,0.000000,0.0,0.0,0


In [9]:
%%sql
select * from plan_events limit 5

,plan_id,project_id__hashed,plan_created_at,event_type,event_timestamp,plan_name,network_provider_id,price_currency,plan_price_amount_local,data_allowance_mb,is_unlimited_data,voice_allowance_seconds,is_unlimited_voice,sms_allowance,is_unlimited_sms,validity_value,validity_unit,_valid_from,_valid_to,_is_current_state
0,pln_b2a155624e05108d1a5b6b90f567,82728d5d3cf7f35b58bc318399c2c5caf7eeadcc37f359...,2023-12-29 18:43:12,plan.updated,2023-12-29 18:44:37.573698,Unlimited Data Plan,p4,USD,70.0,NaN,True,NaN,True,NaN,True,30,day,2023-12-29 18:44:37.573698,2023-12-29 18:44:37.846803,False
1,pln_b2a155624e05108d1a5b6b90f567,82728d5d3cf7f35b58bc318399c2c5caf7eeadcc37f359...,2023-12-29 18:43:12,plan.archived,2024-01-21 15:48:13.444971,Unlimited Data Plan,p4,USD,70.0,NaN,True,NaN,True,NaN,True,30,day,2024-01-21 15:48:13.444971,NaT,True
2,pln_b2a155624e05108d1a5b6b90f567,82728d5d3cf7f35b58bc318399c2c5caf7eeadcc37f359...,2023-12-29 18:43:12,plan.updated,2024-01-21 15:48:12.953812,Unlimited Data Plan,p4,USD,70.0,NaN,True,NaN,True,NaN,True,30,day,2024-01-21 15:48:12.953812,2024-01-21 15:48:13.444971,False
3,pln_b2a155624e05108d1a5b6b90f567,82728d5d3cf7f35b58bc318399c2c5caf7eeadcc37f359...,2023-12-29 18:43:12,plan.published,2023-12-29 18:44:37.846803,Unlimited Data Plan,p4,USD,70.0,NaN,True,NaN,True,NaN,True,30,day,2023-12-29 18:44:37.846803,2024-01-21 15:48:12.953812,False
4,pln_b2a155624e05108d1a5b6b90f567,82728d5d3cf7f35b58bc318399c2c5caf7eeadcc37f359...,2023-12-29 18:43:12,plan.created,2023-12-29 18:43:12.010884,Unlimited Data Plan,p4,USD,70.0,NaN,True,NaN,True,NaN,True,30,day,2023-12-29 18:43:12.000000,2023-12-29 18:44:37.573698,False


In [10]:
%%sql
select * from projects limit 5

,project_id__hashed,project_type,organization_name,device_type
0,dace2786aee7632e61757b320a6fe5bff37a2e742fe558...,API,People Mobile,Phones
1,82728d5d3cf7f35b58bc318399c2c5caf7eeadcc37f359...,Connect,ACME Phone,Phones
2,2aeca1a6c1ecf52b28b7f7646b6fb90563a417ee3e5dc3...,Connect,SmartDevices Inc.,Wearables
